In [1]:
import pandas as pd
import json
import requests

In [2]:
x_rapidapi_key = "90449aaa35msh26907a08c8aecdcp1b16f5jsncf802a2cc113"

Now that we know what JSON is and we have connected to the skyscanner API let's work with it. In this lesson we will create a function for which if we introduce two dates, an origin and a destination we will get the data from the cheapest flight between these two dates.

We will start by creating a function that takes two dates and return a list of dates between the given ones. Luckily, there is a pandas method for that.

In [3]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) # Format makes the output ans str variable.

get_dates('1992-01-01','1992-01-20')

0     1992-01-01
1     1992-01-02
2     1992-01-03
3     1992-01-04
4     1992-01-05
5     1992-01-06
6     1992-01-07
7     1992-01-08
8     1992-01-09
9     1992-01-10
10    1992-01-11
11    1992-01-12
12    1992-01-13
13    1992-01-14
14    1992-01-15
15    1992-01-16
16    1992-01-17
17    1992-01-18
18    1992-01-19
19    1992-01-20
dtype: object

In [4]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [5]:
city_code('London')

'LOND-sky'

Let's use the funtion to get the flights between two destinations on a given date.

In [6]:
def flight_prices(departure, arrival, date):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

flight_prices('Paris', 'London','2020-12-10')

{'Quotes': [{'QuoteId': 1,
   'MinPrice': 69,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [881],
    'OriginId': 44759,
    'DestinationId': 65698,
    'DepartureDate': '2020-12-10T00:00:00'},
   'QuoteDateTime': '2020-11-29T11:28:00'}],
 'Carriers': [{'CarrierId': 881, 'Name': 'British Airways'}],
 'Places': [{'Name': 'Paris Charles de Gaulle',
   'Type': 'Station',
   'PlaceId': 44759,
   'IataCode': 'CDG',
   'SkyscannerCode': 'CDG',
   'CityName': 'Paris',
   'CityId': 'PARI',
   'CountryName': 'France'},
  {'Name': 'London Heathrow',
   'Type': 'Station',
   'PlaceId': 65698,
   'IataCode': 'LHR',
   'SkyscannerCode': 'LHR',
   'CityName': 'London',
   'CityId': 'LOND',
   'CountryName': 'United Kingdom'},
  {'Name': 'Paris',
   'Type': 'City',
   'PlaceId': 3579769,
   'SkyscannerCode': 'PARI',
   'CityId': 'PARI',
   'CountryName': 'France'}],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': Tr

Now that we already have our date function let's create another function that takes the origin, destination and dates as arguments. It will create a dictionary that has every day between the two dates and containts a call to the API with all the information about the days:

In [7]:
from  time import sleep

def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    my_dict = {}
    for date in list(dates):

        try: 

            result = flight_prices(origin, destination, date)
            my_dict[date] = result
            sleep(5)
        except: 
            print("Error from the API")
            sleep(60)
    return my_dict

#flight_days('Paris','Barcelona','2020-11-26','2020-12-10')

In [12]:
flights = flight_days('Barcelona','London','2020-12-26','2021-01-10')

In [14]:
flights

{'2020-12-26': {'Quotes': [{'QuoteId': 1,
    'MinPrice': 41,
    'Direct': False,
    'OutboundLeg': {'CarrierIds': [1090],
     'OriginId': 42414,
     'DestinationId': 82398,
     'DepartureDate': '2020-12-26T00:00:00'},
    'QuoteDateTime': '2020-11-29T11:46:00'},
   {'QuoteId': 2,
    'MinPrice': 42,
    'Direct': True,
    'OutboundLeg': {'CarrierIds': [1865],
     'OriginId': 42414,
     'DestinationId': 65655,
     'DepartureDate': '2020-12-26T00:00:00'},
    'QuoteDateTime': '2020-11-29T07:59:00'}],
  'Carriers': [{'CarrierId': 1865, 'Name': 'Vueling Airlines'},
   {'CarrierId': 1090, 'Name': 'Ryanair'}],
  'Places': [{'Name': 'Barcelona',
    'Type': 'Station',
    'PlaceId': 42414,
    'IataCode': 'BCN',
    'SkyscannerCode': 'BCN',
    'CityName': 'Barcelona',
    'CityId': 'BARC',
    'CountryName': 'Spain'},
   {'Name': 'London Gatwick',
    'Type': 'Station',
    'PlaceId': 65655,
    'IataCode': 'LGW',
    'SkyscannerCode': 'LGW',
    'CityName': 'London',
    'CityId':

Now let's create another function that returns the date of the cheapest flight.

In [16]:
def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)

# flights = flight_prices('Barcelona','London','2020-11-26','2020-12-10')

# my_dict = {date1: [], date2: [],....}

# my_dict = {}


lowest_price('Barcelona','London','2020-12-26','2021-01-10')


'2020-12-27'